In [113]:
import pandas as pd

In [219]:
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.metrics import root_mean_squared_error, r2_score

from sklearn import pipeline

In [115]:
data = pd.read_csv("test_merge_data.csv", index_col=0)
data.head()

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement
1,ID86355,2025-09-10 09:30:05.000,2025-09-10 09:30:05.103,MWYN,1,225,2.65,2.65,ID1516,MWYN,2.61,2.65,5.0,2.0,11.0,8.0,2025-09-10 09:30:04.181625568,0.0
2,ID86359,2025-09-10 09:30:05.962,2025-09-10 09:33:37.670,CDTG,1,1000,1.30,1.30,ID1516,CDTG,1.30,1.31,45.0,19.0,8.0,11.0,2025-09-10 09:30:05.736571594,0.0
4,ID86369,2025-09-10 09:30:07.228,2025-09-10 09:37:43.187,AEHL,1,100,12.00,12.00,ID1516,AEHL,13.33,13.64,1.0,1.0,8.0,11.0,2025-09-10 09:30:06.628618340,0.0
5,ID86382,2025-09-10 09:30:10.105,2025-09-10 09:30:10.210,CUPR,2,89,2.93,3.03,ID1516,CUPR,3.03,3.03,1.0,3.0,12.0,11.0,2025-09-10 09:30:10.091706974,0.1
6,ID86390,2025-09-10 09:30:11.890,2025-09-10 09:30:25.795,CUPR,2,53,3.05,3.05,ID1516,CUPR,3.02,3.03,9.0,1.0,11.0,11.0,2025-09-10 09:30:11.866494533,0.0


In [193]:
data.reset_index(inplace=True)
data.drop(["index"], axis=1, inplace=True)
data.side = data.side.apply(lambda x:x if x==1 else 0)
data.head()

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement
0,ID86355,2025-09-10 09:30:05.000,2025-09-10 09:30:05.103,MWYN,1,225,2.65,2.65,ID1516,MWYN,2.61,2.65,5.0,2.0,11.0,8.0,2025-09-10 09:30:04.181625568,0.0
1,ID86359,2025-09-10 09:30:05.962,2025-09-10 09:33:37.670,CDTG,1,1000,1.30,1.30,ID1516,CDTG,1.30,1.31,45.0,19.0,8.0,11.0,2025-09-10 09:30:05.736571594,0.0
2,ID86369,2025-09-10 09:30:07.228,2025-09-10 09:37:43.187,AEHL,1,100,12.00,12.00,ID1516,AEHL,13.33,13.64,1.0,1.0,8.0,11.0,2025-09-10 09:30:06.628618340,0.0
3,ID86382,2025-09-10 09:30:10.105,2025-09-10 09:30:10.210,CUPR,0,89,2.93,3.03,ID1516,CUPR,3.03,3.03,1.0,3.0,12.0,11.0,2025-09-10 09:30:10.091706974,0.1
4,ID86390,2025-09-10 09:30:11.890,2025-09-10 09:30:25.795,CUPR,0,53,3.05,3.05,ID1516,CUPR,3.02,3.03,9.0,1.0,11.0,11.0,2025-09-10 09:30:11.866494533,0.0


In [117]:
data.groupby("exchange").count()

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement
exchange,,,,,,,,,,,,,,,,,
ID1516,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179,33179
ID245333,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
ID295386,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
ID29608,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60
ID412967,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [195]:
ex1 = data.loc[data.exchange == "ID1516"]
ex2 = data.loc[data.exchange == "ID29608"]

In [196]:
numeric_features = ["order_qty", "limit_price", "bid_price", "ask_price", "bid_size", "ask_size"]
#we adjusted 'side' to be a binary feature for 'side'=="B"
binary_feature = ["side"]

X = ex1[binary_feature + numeric_features]
y = ex1["price_improvement"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=731)


preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    ("passthrough", binary_feature)
)
preprocessor.fit(X_train)
X_train_scaled = preprocessor.transform(X_train)
X_test_scaled  = preprocessor.transform(X_test)




In [197]:
#use linear regression as a baseline:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
preds = model.predict(X_test_scaled)
rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print([rmse, r2])



[0.14704585515866248, -0.020939244423129555]


In [198]:
#lets start temp checking a small list of models. hyperparameter tuning to come later
models = {"LinearRegression": LinearRegression(),
          "Ridge": Ridge(), "Lasso": Lasso(),
          "ElasticNet": ElasticNet(),
          "RandomForest": RandomForestRegressor(),
          "ExtraTrees": ExtraTreesRegressor(),
          "GradientBoosting": GradientBoostingRegressor()}

results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

pd.DataFrame(results, index=["RMSE", "R2"]).T

,RMSE,R2
LinearRegression,0.147046,-0.020939
Ridge,0.145464,0.000906
Lasso,0.145530,-0.000005
ElasticNet,0.145530,-0.000005
RandomForest,0.140683,0.065498
ExtraTrees,0.122682,0.289346
GradientBoosting,0.139927,0.075527


In [199]:
#try adding a power transformer

numeric_pipeline = pipeline.Pipeline([
    ("power", PowerTransformer()),
    ("scale", StandardScaler())
])

preprocessor = make_column_transformer(
    (numeric_pipeline, numeric_features),
    ("passthrough", binary_feature)
)
preprocessor.fit(X_train)
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed  = preprocessor.transform(X_test)

results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_preprocessed, y_train)
    preds = model.predict(X_test_preprocessed)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

pd.DataFrame(results, index=["RMSE", "R2"]).T

,RMSE,R2
LinearRegression,0.144432,0.015035
Ridge,0.144376,0.015794
Lasso,0.145530,-0.000005
ElasticNet,0.145530,-0.000005
RandomForest,0.138652,0.092296
ExtraTrees,0.141474,0.054968
GradientBoosting,0.139959,0.075098


In [200]:
#try not exempting the binary variable from the scaler
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (StandardScaler(), binary_feature)
)
preprocessor.fit(X_train)
X_train_all_scaled = preprocessor.transform(X_train)
X_test_all_scaled  = preprocessor.transform(X_test)

results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_all_scaled, y_train)
    preds = model.predict(X_test_all_scaled)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

pd.DataFrame(results, index=["RMSE", "R2"]).T



,RMSE,R2
LinearRegression,0.147046,-0.020939
Ridge,0.145464,0.000906
Lasso,0.145530,-0.000005
ElasticNet,0.145530,-0.000005
RandomForest,0.140395,0.069321
ExtraTrees,0.121364,0.304533
GradientBoosting,0.141038,0.060783


In [201]:
#try adding a power transformer and scaling the binary

numeric_pipeline = pipeline.Pipeline([
    ("power", PowerTransformer()),
    ("scale", StandardScaler())
])

preprocessor = make_column_transformer(
    (numeric_pipeline, numeric_features),
    (StandardScaler(), binary_feature)
)
preprocessor.fit(X_train)
X_train_all_preprocessed = preprocessor.transform(X_train)
X_test_all_preprocessed  = preprocessor.transform(X_test)

results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_all_preprocessed, y_train)
    preds = model.predict(X_test_all_preprocessed)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

pd.DataFrame(results, index=["RMSE", "R2"]).T

,RMSE,R2
LinearRegression,0.144432,0.015035
Ridge,0.144376,0.015794
Lasso,0.145530,-0.000005
ElasticNet,0.145530,-0.000005
RandomForest,0.138544,0.093698
ExtraTrees,0.140486,0.068125
GradientBoosting,0.140430,0.068861


In [202]:
#let's add a grid search to the different models
#the best performers were randomforest, extratrees, and gradientboosting
#use the preprocessing for each that worked best

models = {
    "RandomForest": RandomForestRegressor(n_jobs=-1, random_state=731),
    "ExtraTrees": ExtraTreesRegressor(n_jobs=-1, random_state=731),
    "GradientBoosting": GradientBoostingRegressor(),
}

param_grids = {
    "RandomForest": {
        "n_estimators": [100, 300],
        "min_samples_leaf": [1, 3],
        "max_features": ["sqrt", "log2", 1],
    },

    "ExtraTrees": {
        "n_estimators": [100, 300],
        "min_samples_leaf": [1, 3],
        "max_features": ["sqrt", "log2", 1],
    },

    "GradientBoosting": {
        "n_estimators": [100, 300],
        "learning_rate": [0.05, 0.1],
        "max_depth": [2, 3],
    },
}

preprocessing = {"RandomForest": (X_train_all_preprocessed, X_test_all_preprocessed),
                 "ExtraTrees": (X_train_all_scaled, X_test_all_scaled),
                 "GradientBoosting": (X_train_scaled, X_test_scaled)}

results = {}

for name, model in models.items():

    X_train_best, X_test_best = preprocessing[name]

    print(f"Grid search for {name}...")

    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        cv=5,
        scoring="neg_root_mean_squared_error",
    )

    gs.fit(X_train_best, y_train)

    best_model = gs.best_estimator_
    preds = best_model.predict(X_test_best)

    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    results[name] = {
        "best_params": gs.best_params_,
        "rmse": rmse,
        "r2": r2,
    }


Grid search for RandomForest...
Grid search for ExtraTrees...
Grid search for GradientBoosting...


In [203]:
results_df = pd.DataFrame(results).T
results_df

,best_params,rmse,r2
RandomForest,"{'max_features': 'sqrt', 'min_samples_leaf': 1...",0.136806,0.116296
ExtraTrees,"{'max_features': 'sqrt', 'min_samples_leaf': 1...",0.125229,0.259534
GradientBoosting,"{'learning_rate': 0.05, 'max_depth': 2, 'n_est...",0.140406,0.069184


In [204]:
chosen_model = results_df.rmse.idxmin()
chosen_params = results_df.best_params[chosen_model]
print(chosen_model)
print(chosen_params)

ExtraTrees
{'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 100}


In [205]:
#now we repeat the process for exchange 2

X = ex2[binary_feature + numeric_features]
y = ex2["price_improvement"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=731)


#since this is a very small dataset (n=60), we will include some models that are more time intensive
#but work well for small datasets.


models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor(n_jobs=-1, random_state=731),
    "ExtraTrees": ExtraTreesRegressor(n_jobs=-1, random_state=731),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR(kernel="rbf"),
    "GaussianProcess": GaussianProcessRegressor(random_state=731)
}

In [206]:
#first, only scale numeric variables

preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    ("passthrough", binary_feature)
)
preprocessor.fit(X_train)
X_train_scaled = preprocessor.transform(X_train)
X_test_scaled  = preprocessor.transform(X_test)

In [207]:
#use linear regression as a baseline:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
preds = model.predict(X_test_scaled)
rmse = root_mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print([rmse, r2])



[0.14074834837907557, 0.03337112319326485]


In [208]:
results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

pd.DataFrame(results, index=["RMSE", "R2"]).T

,RMSE,R2
LinearRegression,0.140748,0.033371
Ridge,0.142231,0.012900
Lasso,0.147515,-0.061811
ElasticNet,0.147515,-0.061811
RandomForest,0.150378,-0.103422
ExtraTrees,0.103238,0.479942
GradientBoosting,0.154303,-0.161780
SVR,0.162477,-0.288125
GaussianProcess,0.118396,0.316010


In [209]:
#try adding a power transformer

numeric_pipeline = pipeline.Pipeline([
    ("power", PowerTransformer()),
    ("scale", StandardScaler())
])

preprocessor = make_column_transformer(
    (numeric_pipeline, numeric_features),
    ("passthrough", binary_feature)
)
preprocessor.fit(X_train)
X_train_preprocessed = preprocessor.transform(X_train)
X_test_preprocessed  = preprocessor.transform(X_test)

results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_preprocessed, y_train)
    preds = model.predict(X_test_preprocessed)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

pd.DataFrame(results, index=["RMSE", "R2"]).T

,RMSE,R2
LinearRegression,0.141860,0.018046
Ridge,0.148691,-0.078797
Lasso,0.147515,-0.061811
ElasticNet,0.147515,-0.061811
RandomForest,0.150374,-0.103364
ExtraTrees,0.128483,0.194506
GradientBoosting,0.153232,-0.145700
SVR,0.151168,-0.115050
GaussianProcess,0.146481,-0.046970


In [210]:
#try not exempting the binary variable from the scaler
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    (StandardScaler(), binary_feature)
)
preprocessor.fit(X_train)
X_train_all_scaled = preprocessor.transform(X_train)
X_test_all_scaled  = preprocessor.transform(X_test)

results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_all_scaled, y_train)
    preds = model.predict(X_test_all_scaled)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

pd.DataFrame(results, index=["RMSE", "R2"]).T



,RMSE,R2
LinearRegression,0.140748,0.033371
Ridge,0.142049,0.015429
Lasso,0.147515,-0.061811
ElasticNet,0.147515,-0.061811
RandomForest,0.150378,-0.103422
ExtraTrees,0.103238,0.479942
GradientBoosting,0.151000,-0.112568
SVR,0.161710,-0.275989
GaussianProcess,0.117987,0.320730


In [211]:
#try adding a power transformer and scaling the binary

numeric_pipeline = pipeline.Pipeline([
    ("power", PowerTransformer()),
    ("scale", StandardScaler())
])

preprocessor = make_column_transformer(
    (numeric_pipeline, numeric_features),
    (StandardScaler(), binary_feature)
)
preprocessor.fit(X_train)
X_train_all_preprocessed = preprocessor.transform(X_train)
X_test_all_preprocessed  = preprocessor.transform(X_test)

results = {}

for model_name in models:
    model = models[model_name]
    model.fit(X_train_all_preprocessed, y_train)
    preds = model.predict(X_test_all_preprocessed)
    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results[model_name] = [rmse, r2]

pd.DataFrame(results, index=["RMSE", "R2"]).T

,RMSE,R2
LinearRegression,0.141860,0.018046
Ridge,0.148516,-0.076262
Lasso,0.147515,-0.061811
ElasticNet,0.147515,-0.061811
RandomForest,0.150374,-0.103364
ExtraTrees,0.128483,0.194506
GradientBoosting,0.159617,-0.243166
SVR,0.152566,-0.135758
GaussianProcess,0.145023,-0.026232


In [220]:
#gaussianprocess and extratrees work best. we will grid search those to optimize hyperparameters
#we can use leave one out cross validation since the dataset is quite small


#import some kernels to try out on the gaussian process model
from sklearn.gaussian_process.kernels import (
    RBF, Matern, RationalQuadratic, ConstantKernel
)

k_rbf = ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(length_scale=1.0, length_scale_bounds="fixed")
k_rbf_long = ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(length_scale=5.0, length_scale_bounds="fixed")
k_matern = ConstantKernel(1.0, constant_value_bounds="fixed") * Matern(length_scale=1.0, nu=1.5, length_scale_bounds="fixed")
k_rq = ConstantKernel(1.0, constant_value_bounds="fixed") * RationalQuadratic(alpha=1.0, length_scale=1.0)


models = {
    "ExtraTrees": ExtraTreesRegressor(n_jobs=-1, random_state=731),
    "GaussianProcess": GaussianProcessRegressor(optimizer=None, normalize_y=True, n_restarts_optimizer=5, random_state=731)
}

param_grids = {"ExtraTrees": {
    "n_estimators": [100, 300],
    "min_samples_leaf": [1, 3],
    "max_features": ["sqrt", "log2", 1],
    },

    "GaussianProcess": {
    "kernel": [
        None,
        k_rbf,
        k_rbf_long,
        k_matern,
        k_rq,
    ],
    "alpha": [1e-10, 1e-6, 1e-4, 1e-2],
}
    }

preprocessing = {"ExtraTrees": (X_train_scaled, X_test_scaled),
                 "GaussianProcess": (X_train_all_scaled, X_test_all_scaled)}

results = {}

for name, model in models.items():
    print(f"Grid search for {name}...")

    X_train_best, X_test_best = preprocessing[name]

    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grids[name],
        cv=LeaveOneOut(),
        scoring="neg_root_mean_squared_error",
    )

    gs.fit(X_train_best, y_train)

    best_model = gs.best_estimator_
    preds = best_model.predict(X_test_best)

    rmse = root_mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    results[name] = {
        "best_params": gs.best_params_,
        "rmse": rmse,
        "r2": r2,
    }


Grid search for ExtraTrees...
Grid search for GaussianProcess...


In [221]:
results_df = pd.DataFrame(results).T
results_df

,best_params,rmse,r2
ExtraTrees,"{'max_features': 'sqrt', 'min_samples_leaf': 1...",0.125443,0.232172
GaussianProcess,"{'alpha': 1e-06, 'kernel': 1**2 * Matern(lengt...",0.147213,-0.057464


In [222]:
chosen_model = results_df.rmse.idxmin()
chosen_params = results_df.best_params[chosen_model]
print(chosen_model)
print(chosen_params)

ExtraTrees
{'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 100}


In [189]:
from somewhat_smart_order_router import best_price_improvement

In [192]:
data.groupby("side").count()

,order_id,order_time,execution_time,symbol,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement
side,,,,,,,,,,,,,,,,,
1,18882,18882,18882,18882,18882,18882,18882,18882,18882,18882,18882,18882,18882,18882,18882,18882,18882
2,14032,14032,14032,14032,14032,14032,14032,14032,14032,14032,14032,14032,14032,14032,14032,14032,14032
5,333,333,333,333,333,333,333,333,333,333,333,333,333,333,333,333,333


In [191]:
best_price_improvement("SQQQ", 2, 28504, 16.91, 16.9, 16.91, 669, 653)

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/models/model/'